In [72]:
import numpy as np
from scipy import fftpack
import plotly.express as px           # Interactive plots
import scipy.signal as signal
import matplotlib.pyplot as plt

In [154]:
Fs = 2756     # the sampling frequency
F_plot = 2756  # the frequency used for plotting the time-continuous curves
T = 0.1       # the time-span we'll cover
t = np.arange(0, T, 1/Fs)  # the sample times
t_plot = np.arange(0, t.max(), 1/F_plot)  # time instants for plotting
n = 1     # Number of harmonics, 1 default
# f1 = 2.5
# f2 = min(f1, Fs-f1)  # determine the alias frequency

xt1 = lambda t: np.sin(2*np.pi*n*f1*t) # create both sine-functions
xt2 = lambda t: np.sin(2*np.pi*n*f2*t)

def showAlias(f1):
    plt.gcf().clear()
    
    # plot the signals
    plt.subplot(121)
    plt.plot(t_plot, xt1(t_plot), 'b-', lw=2, label='input signal')
    plt.stem(t, xt1(t), label='sampled points')
    plt.plot(t_plot, xt2(t_plot), 'g-', label='after sampling')
    plt.ylim((-1.1, 1.5)); plt.grid(True)
    plt.legend(fontsize=8)
    plt.xlabel('$t$'); plt.ylabel('$x(t), x[n]$')
    
    # plot the spectrum of the signals
    t_freq = np.arange(0, 20*T, 1/F_plot) 
    x1 = xt1(t_freq)
    x2 = xt2(t_freq)
    X1 = np.fft.fftshift(np.fft.fft(x1, 8*len(x1))) / len(x1)
    X2 = np.fft.fftshift(np.fft.fft(x2, 8*len(x1))) / len(x2)
    f = np.linspace(-F_plot/2, F_plot/2, len(X1), endpoint=False)
    plt.subplot(122)
    plt.plot(f, abs(X1), lw=2, label='input')
    plt.plot(f, abs(X2), label='after sampling')
    plt.legend(loc='upper left', fontsize=8)
    plt.xlim((-Fs, Fs))
    plt.axvline(-Fs/2, color='k', ls='--', lw=2)
    plt.axvline(Fs/2, color='k', ls='--', lw=2)
    plt.ylim((-0.1, 1.1))
    plt.grid(True)
    plt.text(x=2.5, y=0.8, s='$f_{in}=%.2f$\n$f_{out}=%.2f$' % (f1, f2), bbox=dict(facecolor='white'))
    plt.xlabel('$f$'); plt.ylabel('$|X(f)|$')
    plt.show()

In [155]:
def harmonicWaves(harmonics, f1):
    finalWav = 0
    amp = 1
    sW = lambda t: amp*np.sin(2*np.pi*(n+1)*f1*t)
    for n in range(harmonics):
        nWav = sW(t)
        finalWav += nWav
        amp = amp * 0.9
    
    return finalWav
        

In [156]:
# Adding noise using target SNR
def addNoise(data):
    # Set a target SNR
    target_snr_db = 10
    # Calculate signal power and convert to dB 
    sig_avg_watts = np.mean(sinE ** 2)
    sig_avg_db = 10 * np.log10(sig_avg_watts)
    # Calculate noise according to [2] then convert to watts
    noise_avg_db = sig_avg_db - target_snr_db
    noise_avg_watts = 10 ** (noise_avg_db / 10)
    # Generate an sample of white noise
    mean_noise = 0
    noise = np.random.normal(mean_noise, np.sqrt(noise_avg_watts), len(sinE))
    # Noise up the original signal
    return data + noise

In [157]:
def AAF(data, Fc):
    Fc = 2*Fc/Fs   # Fs = 2756
#     print(Fc)
#     print(Fc, Fs)
    b = signal.firwin(20001, Fc); a=1  # design the AAF
#     b ,a= signal.butter(499, Fc, 'low', fs = Fs); a=1
    lowpass = signal.lfilter(b, a, data)      # apply the AAF
    return lowpass
    

In [166]:
sinE = harmonicWaves(6, 440)
sinENoise = addNoise(sinE)
# print(Fs)

FFTsinE = fftpack.fft(sinENoise)          # FFT to get frequency spectrum
FFTsinE = FFTsinE[:len(FFTsinE)//2]  # To remove repeated spectra
N = len(FFTsinE)
freqArray = [x*(Fs/2)/N for x in list(range(0, N))] # Y axis frequencies

sinEAAF = AAF(sinENoise, 1377)
# print(sinEAAF)
sinEAAF = sinEAAF[:len(sinEAAF)//2]


In [167]:
# Plot the unfiltered raw data 
figure = px.line(x=t, y=sinENoise, labels = {'x':'Time (s)', 'y':'Amplitude'}, title='Unfiltered Data', width=750)
figure.show()

# Plotting unfiltered data signal spectrum

# Non-Log
# figure = px.line(x=freqArray, y=abs(FFTsinE), labels = {'x':'Freq (Hz)', 'y':'Voltage (μV)'}, title='Unfiltered Frequency Spectrum', width=750)
# figure.show()

# Log
figure = px.line(x=freqArray, y=20*np.log10(abs(FFTsinE)), labels = {'x':'Freq (Hz)', 'y':'Magnitude (dB)'}, title='Unfiltered Frequency Spectrum', width=750)
figure.show()

print(len(freqArray))
# print(len(20*np.log10(abs(sinEAAF))))

figure = px.line(x=freqArray, y= 20*np.log10(abs(sinEAAF)), labels = {'x':'Freq (Hz)', 'y':'Magnitude (dB)'}, title='nfiltered Frequency Spectrum', width=750)
figure.show()

138


In [130]:
print(1350 < Fs/2)

True


In [13]:
plt.figure()
interact(showAlias, f1=(0,5., 0.01));

NameError: name 'plt' is not defined